In [ ]:
# Uncomment this cell if it's your first time running spacy
#pip install -U spacy
#!python -m spacy download en_core_web_sm
#!python -m spacy download zh_core_web_sm
#!python -m spacy download xx_sent_ud_sm

In [27]:
import pandas as pd
import numpy as np
import os
import jieba
import string 
import re 
from nltk.tokenize import MWETokenizer, word_tokenize
from tqdm import tqdm
import spacy

In [2]:
language_pairs = [
    'en-fi', 'en-zh', 'cs-en'
    , 'de-en', 'ru-en', 'zh-en'
]
translations = ['fi', 'zh', 'en']

In [3]:
corpus_fi = []
corpus_zh = []
corpus_en = []
for pair in language_pairs:
    languages = pair.split('-')
    for i in range(2):
        all_sentences = pd.read_csv(os.path.join('..', 'corpus',pair, 'scores.csv'))
        if i == 0:
            sentences = all_sentences['source']
        if i == 1:
            sentences = all_sentences['reference'].append(all_sentences['translation'], ignore_index=True)
              
        if languages[i]=='fi':
            corpus_fi.append(sentences)
        elif languages[i]=='zh':
            corpus_zh.append(sentences)
        elif languages[i]=='en':
            corpus_en.append(sentences)


In [4]:
corpus_fi = list(pd.concat(corpus_fi))
corpus_fi

['Voit muuttaa itsesi ananasta, koirasta tai Roy Lichtensteinin maalauksesta sopivasta merkistä.',
 'Myös ammuttiin kolme miestä: kaksi 29-vuotiaita ja yksi 32-vuotias.',
 'Tiedot tallennetaan kassakoneisiin joka tapauksessa, haluatko sen vai ei.',
 'Xinhua kertoo, että Xinyin näytteestä oli sunnuntaina hydroklooritiatsididiureettia.',
 'Voitaisiin kuulla CBD: n kommenttitiimin toimittama MacDonald, joka sanoi: "Se pieni 14-vuotias Kiinasta pudotti pallon, vauva".',
 'Varastettujen polkupyörän ostajat ovat tavanomaisia suomalaisia, poliisin mukaan.',
 'Salomon Koli, joka nostaa naudanlihasta, pyrkii investoimaan entistä suurempaan lehmähauteen.',
 'Uutiset ja yliluonnos menivät lauantaina keskiviikkona, The Vergen ja emolevyn raportit, päivä sen jälkeen, kun Snapchat sanoi, että se oli purkanut ominaisuuden.',
 'Viimeisin menetelmäuudistus',
 'Tytär, jonka hänen isovanhempansa käyttivät taapuisena ja jonka valmentaja Aimee Boormanin äiti löysi kenttämatkalle kuntosalille, jossa Boorman

In [5]:
corpus_en = list(pd.concat(corpus_en))
corpus_en

['You can turn yourself into a pineapple, a dog or a character befitting a Roy Lichtenstein painting.',
 'Also shot were three men: two 29-year-olds and one 32-year old.',
 'The information is stored at the cash registers anyway, whether you want it or not.',
 'Xinhua says that there were traces of hydrochlorothiazide diuretic in Xinyi’s sample from Sunday.',
 "MacDonald, who was brought on board CBC's commentary team to provide 'color' could be heard saying: 'That little 14-year-old from China dropped the ball, baby'.",
 'Buyers of stolen bikes are normal Finnish people, according to the police.',
 'Salomon Koli, who raises beef cattle, has a goal of investing in even bigger a cowshed.',
 'The news and the outrage went wide on Wednesday, with reports by The Verge and Motherboard, a day after Snapchat said it had dismantled the feature.',
 'The most recent method reform',
 "The girl adopted by her grandparents as a toddler and discovered by coach Aimee Boorman's mother during a field t

In [6]:
corpus_zh = list(pd.concat(corpus_zh))
corpus_zh

['GSIS的科学家AnthonyDelGenio在新闻稿中解释说：“在GISS模型的模拟模型中，金星的缓慢使它在太阳下了两个月。',
 '中国在英国女性4x200mFreestreyWTE中的最后被称为：“中国14岁的孩子从球下降到了孩子。”',
 '然后来到2012年，当她和她的队友们没有什么好处。',
 '自去年以来，GoudianGroup从南非通过南非港口出口了163套风力发电项目。',
 '一些人指称，Kempinski旅馆只是"被捕"，以满足阿拉伯客户的要求。',
 '他回忆说，在Binbin一年后进入厦门射击球队后，队伍开展了一个长时间的枪支训练，要求玩家不要半小时。',
 '自从1923年至1943年，他们在家庭经济学家，',
 '对于老朋友来说，汉宁希望他们更好地站起来，特别是心血管疾病。',
 '“他找到了一个公寓，他和约会一个女孩，”路易斯·卡利亚说。',
 '斐济高兴地发动了一个有趣的事.',
 '里德说，克林顿将如果被选举产生，最高法院提名',
 '据报道，国旗的预计将在11届达到里约，奥林匹克组织委员会在接受旗帜之后将旗帜送到各种奥林匹克场馆，整个过程将受到中国驻巴西大使馆的监督。',
 '另一些观察者也分享她的看法,这是一个关于Spanchat的投诉.',
 '你们代表CBC和加拿大，使我们大家都像一群魔鬼一样。',
 '波特兰冶炼厂的未来可能会影响力量合同',
 '日本问题专家杨洁篪表示：“在政治方面，非洲相对滞后，但是有很多非洲国家，因此日本始终认为它是所谓的“大型售票场”。',
 '根据知情消息来源，最近，Suning高管和苹果经常开会，目的是准备足够的供应。',
 '50岁的罗伊在声明中说：“过去三年来，我已经完成了科罗拉多河Alavanche的乐观、热情和决心副主席和主席。',
 '充足的供应和各种特许性将减轻Phones购买的压力，Suning的后勤将会变得更加直观的经验。',
 '虽然布莱斯坚持她在长期的顶峰期间没有看过，这不是真正的。',
 '女士在瑞士高速公路上运行',
 '尽管俄罗斯和西方没有就叙利亚的极端主义势力达成协议，但俄罗斯将在这方面加强与叙利亚、伊朗和伊拉克的合作。',
 '格陵兰鲨鱼也被称为小睡眠鲨鱼，在格陵兰和冰岛周围的北大西洋水域广泛散发，其成年年龄在4岁到5米。',
 '无论在2017-2018年

In [7]:
def tokenization(corpus, tool, language = 'en', pos_to_remove = ['PUNCT','NUM'], ent_to_remove = ['PERSON','ORG'], stop_words_to_remove= False, lowercase = True):
    """
    tool: one of two strings - 'spacy' or 'NLTK'
    languages (string ISO code): supports 'en', 'fi' or 'zh'
    pos_to_remove (list): part-of-speech tag from spacy
    ent_to_remove (list): entities from spacy
    
    
    """
    tokenized_corpus = []
    if tool == 'spacy':
        if language == 'en':
            sc = spacy.load('en_core_web_sm')
            for doc in tqdm(sc.pipe(corpus, disable=["lemmatizer", "textcat", "custom"])):
                if stop_words_to_remove:
                    doc_list = [word.text for word in doc if word.pos_ not in pos_to_remove if word.ent_type_ not in ent_to_remove if word.is_stop]
                else: 
                    doc_list = [word.text for word in doc if word.pos_ not in pos_to_remove if word.ent_type_ not in ent_to_remove]

                if lowercase:
                    doc_list = [word.lower() for word in doc_list]

                tokenized_corpus.append(doc_list)
        if language == 'zh':
            sc = spacy.load('zh_core_web_sm')
            for doc in tqdm(sc.pipe(corpus, disable=["lemmatizer", "textcat", "custom"])):
                if stop_words_to_remove:
                    doc_list = [word.text for word in doc if word.pos_ not in ['PUNCT'] if word.is_stop]
                else: 
                    doc_list = [word.text for word in doc if word.pos_ not in ['PUNCT']]
                
                tokenized_corpus.append(doc_list)
        if language == 'fi':
            sc = spacy.load('xx_sent_ud_sm')
            for doc in tqdm(sc.pipe(corpus, disable=["lemmatizer", "textcat", "custom"])): 
                doc_list = [word.text for word in doc]
                
                if lowercase:
                    doc_list = [word.lower() for word in doc_list]

                tokenized_corpus.append(doc_list)
                
            
        
    if tool == 'NLTK':
        print('Not implemented')
        
        
    return tokenized_corpus


### Finish Vocabulary

* Spacy:

In [33]:
# all as default ( with punctuation numbers, allll!)
tokenized_fi1 = tokenization(corpus_fi, tool = 'spacy', language = 'fi',lowercase = True)

13496it [00:02, 5362.75it/s]


In [34]:
tokenized_fi1

[['voit',
  'muuttaa',
  'itsesi',
  'ananasta',
  ',',
  'koirasta',
  'tai',
  'roy',
  'lichtensteinin',
  'maalauksesta',
  'sopivasta',
  'merkistä',
  '.'],
 ['myös',
  'ammuttiin',
  'kolme',
  'miestä',
  ':',
  'kaksi',
  '29-vuotiaita',
  'ja',
  'yksi',
  '32-vuotias',
  '.'],
 ['tiedot',
  'tallennetaan',
  'kassakoneisiin',
  'joka',
  'tapauksessa',
  ',',
  'haluatko',
  'sen',
  'vai',
  'ei',
  '.'],
 ['xinhua',
  'kertoo',
  ',',
  'että',
  'xinyin',
  'näytteestä',
  'oli',
  'sunnuntaina',
  'hydroklooritiatsididiureettia',
  '.'],
 ['voitaisiin',
  'kuulla',
  'cbd',
  ':',
  'n',
  'kommenttitiimin',
  'toimittama',
  'macdonald',
  ',',
  'joka',
  'sanoi',
  ':',
  '"',
  'se',
  'pieni',
  '14-vuotias',
  'kiinasta',
  'pudotti',
  'pallon',
  ',',
  'vauva',
  '"',
  '.'],
 ['varastettujen',
  'polkupyörän',
  'ostajat',
  'ovat',
  'tavanomaisia',
  'suomalaisia',
  ',',
  'poliisin',
  'mukaan',
  '.'],
 ['salomon',
  'koli',
  ',',
  'joka',
  'nostaa',
  

In [36]:
def match_regex(tokenized_corpus, letters = True, letters_and_numbers = False):
    
    if letters:
        regex = r'[a-z]+'
    if letters_and_numbers:
        regex = r'([a-z]+|^\d+$)'
        
    new_tokenized_zh = []
    for sentence_list in tokenized_corpus:
        sentence_list2 = [word for word in sentence_list if re.search(regex, word)]
        new_tokenized_zh.append(sentence_list2)
        
    return new_tokenized_zh

In [37]:
# remove punctuation keeping the strings with letters
tokenized_fi2 = match_regex(tokenized_fi1, letters = True, letters_and_numbers = False)

In [38]:
tokenized_fi2

[['voit',
  'muuttaa',
  'itsesi',
  'ananasta',
  'koirasta',
  'tai',
  'roy',
  'lichtensteinin',
  'maalauksesta',
  'sopivasta',
  'merkistä'],
 ['myös',
  'ammuttiin',
  'kolme',
  'miestä',
  'kaksi',
  '29-vuotiaita',
  'ja',
  'yksi',
  '32-vuotias'],
 ['tiedot',
  'tallennetaan',
  'kassakoneisiin',
  'joka',
  'tapauksessa',
  'haluatko',
  'sen',
  'vai',
  'ei'],
 ['xinhua',
  'kertoo',
  'että',
  'xinyin',
  'näytteestä',
  'oli',
  'sunnuntaina',
  'hydroklooritiatsididiureettia'],
 ['voitaisiin',
  'kuulla',
  'cbd',
  'n',
  'kommenttitiimin',
  'toimittama',
  'macdonald',
  'joka',
  'sanoi',
  'se',
  'pieni',
  '14-vuotias',
  'kiinasta',
  'pudotti',
  'pallon',
  'vauva'],
 ['varastettujen',
  'polkupyörän',
  'ostajat',
  'ovat',
  'tavanomaisia',
  'suomalaisia',
  'poliisin',
  'mukaan'],
 ['salomon',
  'koli',
  'joka',
  'nostaa',
  'naudanlihasta',
  'pyrkii',
  'investoimaan',
  'entistä',
  'suurempaan',
  'lehmähauteen'],
 ['uutiset',
  'ja',
  'yliluon

In [13]:
# remove punctuation keeping the strings with letters or only numbers
tokenized_fi3 = match_regex(tokenized_fi1, letters = False, letters_and_numbers = True)

In [14]:
tokenized_fi3

[['voit',
  'muuttaa',
  'itsesi',
  'ananasta',
  'koirasta',
  'tai',
  'roy',
  'lichtensteinin',
  'maalauksesta',
  'sopivasta',
  'merkistä'],
 ['myös',
  'ammuttiin',
  'kolme',
  'miestä',
  'kaksi',
  '29-vuotiaita',
  'ja',
  'yksi',
  '32-vuotias'],
 ['tiedot',
  'tallennetaan',
  'kassakoneisiin',
  'joka',
  'tapauksessa',
  'haluatko',
  'sen',
  'vai',
  'ei'],
 ['xinhua',
  'kertoo',
  'että',
  'xinyin',
  'näytteestä',
  'oli',
  'sunnuntaina',
  'hydroklooritiatsididiureettia'],
 ['voitaisiin',
  'kuulla',
  'cbd',
  'n',
  'kommenttitiimin',
  'toimittama',
  'macdonald',
  'joka',
  'sanoi',
  'se',
  'pieni',
  '14-vuotias',
  'kiinasta',
  'pudotti',
  'pallon',
  'vauva'],
 ['varastettujen',
  'polkupyörän',
  'ostajat',
  'ovat',
  'tavanomaisia',
  'suomalaisia',
  'poliisin',
  'mukaan'],
 ['salomon',
  'koli',
  'joka',
  'nostaa',
  'naudanlihasta',
  'pyrkii',
  'investoimaan',
  'entistä',
  'suurempaan',
  'lehmähauteen'],
 ['uutiset',
  'ja',
  'yliluon

* NLTK:

### English Vocabulary

* Spacy:

In [28]:
# all as default
spacy_tokenized_en1 = tokenization(corpus_en, tool = 'spacy', language = 'en', pos_to_remove = ['PUNCT','NUM'], ent_to_remove = ['PERSON','ORG'], stop_words_to_remove= False, lowercase = True)

172345it [06:43, 426.80it/s]


In [29]:
# not removing entities names
spacy_tokenized_en2 = tokenization(corpus_en, tool = 'spacy', language = 'en', pos_to_remove = ['PUNCT','NUM'], ent_to_remove = [], stop_words_to_remove= False, lowercase = True)

172345it [06:46, 423.62it/s]


In [30]:
# not removing entities names and numbers
spacy_tokenized_en3 = tokenization(corpus_en, tool = 'spacy', language = 'en', pos_to_remove = ['PUNCT'], ent_to_remove = [], stop_words_to_remove= False, lowercase = True)

172345it [06:47, 422.69it/s]


In [31]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
len(spacy_stopwords) #326
list(spacy_stopwords)[:20]

['none',
 'sometime',
 'why',
 'per',
 'much',
 'whereby',
 'others',
 'a',
 'myself',
 'such',
 'anyhow',
 'make',
 'than',
 'within',
 'always',
 'onto',
 'into',
 'until',
 'through',
 'each']

* NLTK:

### Chinese Vocabulary

* Jieba:

In [39]:
jieba_tokenized_zh = []
for sentence in corpus_zh:
    sentence_list = [word for word in jieba.cut(sentence)]
    jieba_tokenized_zh.append(sentence_list)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\betty\AppData\Local\Temp\jieba.cache
Loading model cost 0.612 seconds.
Prefix dict has been built successfully.


In [40]:
def match_regex(tokenized_corpus, chinese = True, chinese_and_numbers = False):
    
    if chinese:
        regex = r'[\u4e00-\u9fff]+'
    if chinese_and_numbers:
        regex = r'([\u4e00-\u9fff]+|^\d+$)'
        
    new_tokenized_zh = []
    for sentence_list in tokenized_corpus:
        chinese = [word for word in sentence_list if re.search(regex, word)]
        new_tokenized_zh.append(chinese)
        
    return new_tokenized_zh

In [41]:
# to keep all words containing chinese characters
jieba_tokenized_zh2 = match_regex(jieba_tokenized_zh, chinese = True, chinese_and_numbers = False)

In [42]:
jieba_tokenized_zh2

[['的',
  '科学家',
  '在',
  '新闻稿',
  '中',
  '解释',
  '说',
  '在',
  '模型',
  '的',
  '模拟',
  '模型',
  '中',
  '金星',
  '的',
  '缓慢',
  '使',
  '它',
  '在',
  '太阳',
  '下',
  '了',
  '两个',
  '月'],
 ['中国',
  '在',
  '英国',
  '女性',
  '中',
  '的',
  '最后',
  '被',
  '称为',
  '中国',
  '岁',
  '的',
  '孩子',
  '从球',
  '下降',
  '到',
  '了',
  '孩子'],
 ['然后', '来到', '年', '当', '她', '和', '她', '的', '队友', '们', '没有', '什么', '好处'],
 ['自',
  '去年',
  '以来',
  '从',
  '南非',
  '通过',
  '南非',
  '港口',
  '出口',
  '了',
  '套',
  '风力',
  '发电',
  '项目'],
 ['一些', '人', '指称', '旅馆', '只是', '被捕', '以', '满足', '阿拉伯', '客户', '的', '要求'],
 ['他',
  '回忆',
  '说',
  '在',
  '一年',
  '后',
  '进入',
  '厦门',
  '射击',
  '球队',
  '后',
  '队伍',
  '开展',
  '了',
  '一个',
  '长时间',
  '的',
  '枪支',
  '训练',
  '要求',
  '玩家',
  '不要',
  '半小时'],
 ['自从', '年', '至', '年', '他们', '在', '家庭', '经济学家'],
 ['对于',
  '老朋友',
  '来说',
  '汉宁',
  '希望',
  '他们',
  '更好',
  '地站',
  '起来',
  '特别',
  '是',
  '心血管',
  '疾病'],
 ['他', '找到', '了', '一个', '公寓', '他', '和', '约会', '一个', '女孩', '路易斯', '卡利亚', '说'],
 ['斐济', '高兴',

In [43]:
# to keep chinese characters and numbers
jieba_tokenized_zh3 = match_regex(jieba_tokenized_zh, chinese = False, chinese_and_numbers = True)

In [44]:
jieba_tokenized_zh3

[['的',
  '科学家',
  '在',
  '新闻稿',
  '中',
  '解释',
  '说',
  '在',
  '模型',
  '的',
  '模拟',
  '模型',
  '中',
  '金星',
  '的',
  '缓慢',
  '使',
  '它',
  '在',
  '太阳',
  '下',
  '了',
  '两个',
  '月'],
 ['中国',
  '在',
  '英国',
  '女性',
  '中',
  '的',
  '最后',
  '被',
  '称为',
  '中国',
  '14',
  '岁',
  '的',
  '孩子',
  '从球',
  '下降',
  '到',
  '了',
  '孩子'],
 ['然后',
  '来到',
  '2012',
  '年',
  '当',
  '她',
  '和',
  '她',
  '的',
  '队友',
  '们',
  '没有',
  '什么',
  '好处'],
 ['自',
  '去年',
  '以来',
  '从',
  '南非',
  '通过',
  '南非',
  '港口',
  '出口',
  '了',
  '163',
  '套',
  '风力',
  '发电',
  '项目'],
 ['一些', '人', '指称', '旅馆', '只是', '被捕', '以', '满足', '阿拉伯', '客户', '的', '要求'],
 ['他',
  '回忆',
  '说',
  '在',
  '一年',
  '后',
  '进入',
  '厦门',
  '射击',
  '球队',
  '后',
  '队伍',
  '开展',
  '了',
  '一个',
  '长时间',
  '的',
  '枪支',
  '训练',
  '要求',
  '玩家',
  '不要',
  '半小时'],
 ['自从', '1923', '年', '至', '1943', '年', '他们', '在', '家庭', '经济学家'],
 ['对于',
  '老朋友',
  '来说',
  '汉宁',
  '希望',
  '他们',
  '更好',
  '地站',
  '起来',
  '特别',
  '是',
  '心血管',
  '疾病'],
 ['他', '找到', '了', '一个', 

* Spacy:

In [45]:
spacy_tokenized_zh = tokenization(corpus_zh, tool = 'spacy', language = 'zh', stop_words_to_remove= False)

46861it [02:20, 332.72it/s]


In [46]:
spacy_tokenized_zh

[['GSIS',
  '的',
  '科学家',
  'Anthony',
  'DelGenio',
  '在',
  '新闻稿',
  '中',
  '解释',
  '说',
  '在',
  'GIS',
  'S模型',
  '的',
  '模拟',
  '模型',
  '中',
  '金星',
  '的',
  '缓慢',
  '使',
  '它',
  '在',
  '太阳',
  '下',
  '了',
  '两',
  '个',
  '月'],
 ['中国',
  '在',
  '英国',
  '女性',
  '4x200m',
  'FreestreyWTE',
  '中',
  '的',
  '最后',
  '被',
  '称为',
  '中国',
  '14',
  '岁',
  '的',
  '孩子',
  '从',
  '球下',
  '降到',
  '了',
  '孩子'],
 ['然后', '来到', '2012年', '当', '她', '和', '她', '的', '队友们', '没有', '什么', '好处'],
 ['自',
  '去年',
  '以来',
  'Goudian',
  'Group',
  '从',
  '南非',
  '通过',
  '南非',
  '港口',
  '出口',
  '了',
  '163',
  '套',
  '风力',
  '发电',
  '项目'],
 ['一些',
  '人',
  '指称',
  'Kempins',
  'ki',
  '旅馆',
  '只是',
  '被',
  '捕"',
  '以',
  '满足',
  '阿拉伯',
  '客户',
  '的',
  '要求'],
 ['他',
  '回忆',
  '说',
  '在',
  'Binbin',
  '一',
  '年后',
  '进入',
  '厦门',
  '射击',
  '球队',
  '后',
  '队伍',
  '开展',
  '了',
  '一个',
  '长',
  '时间',
  '的',
  '枪支',
  '训练',
  '要求',
  '玩家',
  '不要',
  '半',
  '小时'],
 ['自从', '1923年', '至', '1943年', '他们', '在', '家庭', 

In [47]:
spacy_stopwords = spacy.lang.zh.stop_words.STOP_WORDS
len(spacy_stopwords) #1891
list(spacy_stopwords)[:20]

['到头来',
 '不再',
 '咋',
 '反之',
 '这就是说',
 '呐',
 '５：０',
 '/',
 '大略',
 '［⑤］',
 '轰然',
 '一般',
 '据悉',
 '他是',
 '左右',
 '并非',
 '截然',
 '不特',
 '除外',
 '尽量']

In [48]:
# to keep all words containing chinese characters
spacy_tokenized_zh2 = match_regex(spacy_tokenized_zh, chinese = True, chinese_and_numbers = False)

In [49]:
spacy_tokenized_zh2

[['的',
  '科学家',
  '在',
  '新闻稿',
  '中',
  '解释',
  '说',
  '在',
  'S模型',
  '的',
  '模拟',
  '模型',
  '中',
  '金星',
  '的',
  '缓慢',
  '使',
  '它',
  '在',
  '太阳',
  '下',
  '了',
  '两',
  '个',
  '月'],
 ['中国',
  '在',
  '英国',
  '女性',
  '中',
  '的',
  '最后',
  '被',
  '称为',
  '中国',
  '岁',
  '的',
  '孩子',
  '从',
  '球下',
  '降到',
  '了',
  '孩子'],
 ['然后', '来到', '2012年', '当', '她', '和', '她', '的', '队友们', '没有', '什么', '好处'],
 ['自',
  '去年',
  '以来',
  '从',
  '南非',
  '通过',
  '南非',
  '港口',
  '出口',
  '了',
  '套',
  '风力',
  '发电',
  '项目'],
 ['一些', '人', '指称', '旅馆', '只是', '被', '捕"', '以', '满足', '阿拉伯', '客户', '的', '要求'],
 ['他',
  '回忆',
  '说',
  '在',
  '一',
  '年后',
  '进入',
  '厦门',
  '射击',
  '球队',
  '后',
  '队伍',
  '开展',
  '了',
  '一个',
  '长',
  '时间',
  '的',
  '枪支',
  '训练',
  '要求',
  '玩家',
  '不要',
  '半',
  '小时'],
 ['自从', '1923年', '至', '1943年', '他们', '在', '家庭', '经济学家'],
 ['对于',
  '老朋友',
  '来说',
  '汉宁',
  '希望',
  '他们',
  '更',
  '好',
  '地',
  '站',
  '起来',
  '特别是',
  '心血管',
  '疾病'],
 ['他', '找到', '了', '一个', '公寓', '他', '和', '约会', '一个', '

In [50]:
# to keep chinese characters and numbers
spacy_tokenized_zh3 = match_regex(spacy_tokenized_zh, chinese = False, chinese_and_numbers = True)
spacy_tokenized_zh3

[['的',
  '科学家',
  '在',
  '新闻稿',
  '中',
  '解释',
  '说',
  '在',
  'S模型',
  '的',
  '模拟',
  '模型',
  '中',
  '金星',
  '的',
  '缓慢',
  '使',
  '它',
  '在',
  '太阳',
  '下',
  '了',
  '两',
  '个',
  '月'],
 ['中国',
  '在',
  '英国',
  '女性',
  '中',
  '的',
  '最后',
  '被',
  '称为',
  '中国',
  '14',
  '岁',
  '的',
  '孩子',
  '从',
  '球下',
  '降到',
  '了',
  '孩子'],
 ['然后', '来到', '2012年', '当', '她', '和', '她', '的', '队友们', '没有', '什么', '好处'],
 ['自',
  '去年',
  '以来',
  '从',
  '南非',
  '通过',
  '南非',
  '港口',
  '出口',
  '了',
  '163',
  '套',
  '风力',
  '发电',
  '项目'],
 ['一些', '人', '指称', '旅馆', '只是', '被', '捕"', '以', '满足', '阿拉伯', '客户', '的', '要求'],
 ['他',
  '回忆',
  '说',
  '在',
  '一',
  '年后',
  '进入',
  '厦门',
  '射击',
  '球队',
  '后',
  '队伍',
  '开展',
  '了',
  '一个',
  '长',
  '时间',
  '的',
  '枪支',
  '训练',
  '要求',
  '玩家',
  '不要',
  '半',
  '小时'],
 ['自从', '1923年', '至', '1943年', '他们', '在', '家庭', '经济学家'],
 ['对于',
  '老朋友',
  '来说',
  '汉宁',
  '希望',
  '他们',
  '更',
  '好',
  '地',
  '站',
  '起来',
  '特别是',
  '心血管',
  '疾病'],
 ['他', '找到', '了', '一个', '公寓', '他', '